In [9]:
%pip install pydub

/bin/bash: /home/staticowl/miniconda3//lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [10]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# import librosa
# import librosa.display
# import soundfile as sf
# import os

# # Load and preprocess dataset

# # You may need to use librosa or other audio processing libraries for feature extraction

# # Model architecture (Variational Autoencoder for simplicity)
# class VoiceConversionModel(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(VoiceConversionModel, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim, 512),
#             nn.ReLU(),
#             nn.Linear(512, latent_dim),
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(latent_dim, 512),
#             nn.ReLU(),
#             nn.Linear(512, input_dim),
#         )

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return encoded, decoded

# # Define hyperparameters
# input_dim = 128 
# latent_dim = 64  


# model = VoiceConversionModel(input_dim, latent_dim)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 1000
# for epoch in range(num_epochs):
#     inputs = torch.tensor(X_train, dtype=torch.float32)
#     targets = torch.tensor(Y_train, dtype=torch.float32)
#     optimizer.zero_grad()
#     _, outputs = model(inputs)
#     loss = criterion(outputs, targets)
#     loss.backward()
#     optimizer.step()

#     if (epoch + 1) % 100 == 0:
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# input_feature = torch.tensor(X_train[0], dtype=torch.float32)
# _, converted_feature_raw = model(input_feature)
# converted_feature = converted_feature_raw.detach().numpy()
# print("Converted Feature:", converted_feature)



In [11]:
# from pydub import AudioSegment

# def spectrogram_to_audio(spectrogram):
#     # Inverse transform spectrogram to waveform using Griffin-Lim

#     print("Shape of spectrogram:", spectrogram.shape)
#     print("Data type of spectrogram:", spectrogram.dtype)
#     waveform = librosa.griffinlim(np.abs(spectrogram))
#     return waveform

# converted_audio = spectrogram_to_audio(converted_feature)
# sf.write("output.wav", converted_audio, 128)

In [12]:
import tensorflow as tf

W = tf.Variable(tf.random.uniform((2,2)))
b=tf.Variable(tf.zeros((2,)))
x=tf.random.uniform((2,2))

with tf.GradientTape() as tape:
    y=tf.matmul(x,W)+b
grad = tape.gradient(y, [W,b])

print(grad)

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.7537323, 1.7537323],
       [1.2928395, 1.2928395]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 2.], dtype=float32)>]
